In [51]:
#importar librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [52]:

# Cargar el archivo CSV
ruta = r"C:\Users\anaco\OneDrive\Documentos\Master DADS\Proyecto\emocion_valoraciones_con_visitante.csv"
df = pd.read_csv(ruta)
df.head()

,t_id,emocion,fecha_hora,valoracion,procedencia,id_visitante,tipo_entrada,coste,antelacion_de_compra,atraccion,comienzo_atraccion,tiempo_de_espera,comienzo_atraccion_fecha_hora,duracion
0,Training_10118481.jpg,enojado,2022-09-13 10:00:00,2,Filipinas,3,pase anual,14.00,0,Montaña Rusa de la Luna,291.0,5,2022-09-13 10:00:00,308
1,Training_10120469.jpg,enojado,2022-09-19 00:00:00,6,India,4,entrada individual,13.79,113,Carrusel Encantado,425.0,11,2022-09-19 00:00:00,103
2,Training_10131352.jpg,enojado,2022-09-22 02:00:00,0,España,5,entrada infantil,19.06,25,Cine 4D Emocionante,499.0,13,2022-09-22 02:00:00,371
3,Training_10161559.jpg,enojado,2022-10-01 02:00:00,4,España,6,entrada individual,17.72,320,Fiesta de los Dulces,715.0,12,2022-10-01 02:00:00,228
4,Training_1021836.jpg,enojado,2022-09-13 20:00:00,6,Canadá,7,entrada individual,20.50,271,Tobogán del Arco Iris,301.0,17,2022-09-13 20:00:00,247


In [53]:
# Ver tipos de datos y conteo de nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

print("\nTipos de datos:")
print(df.dtypes)

Valores nulos por columna:
t_id                                0
emocion                           351
fecha_hora                          2
valoracion                          0
procedencia                         0
id_visitante                        0
tipo_entrada                        0
coste                               0
antelacion_de_compra                0
atraccion                        1138
comienzo_atraccion                 96
tiempo_de_espera                    0
comienzo_atraccion_fecha_hora      96
duracion                            0
dtype: int64

Tipos de datos:
t_id                              object
emocion                           object
fecha_hora                        object
valoracion                         int64
procedencia                       object
id_visitante                       int64
tipo_entrada                      object
coste                            float64
antelacion_de_compra               int64
atraccion                         object
c

In [32]:
# Eliminar filas sin emoción (target)
df = df.dropna(subset=['emocion'])

# Confirmar que ya no hay nulos en la target
print(df['emocion'].isnull().sum())
print(f"Filas restantes: {len(df)}")

0
Filas restantes: 35536


SEGUNDOS MODELOS MAS COMPLEJOS

ELECCION DE VARIABLES X - JUSTIFICACION

VALORACION : Si puntuó bajo, es más probable que esté enojado, triste o asustado. Si puntuó alto, es más probable que esté feliz o sorprendido
ATRACCION: hay diferencias socioculturales en la percepción del miedo, del asco, o la euforia
TIEMPO ESPERA: una muy larga puede aburrir o generar cansancio.


In [54]:
# Solo eliminamos filas con nulos en las columnas relevantes
df = df[['valoracion', 'atraccion', 'duracion', 'tiempo_de_espera','emocion']].dropna()

In [55]:
# Variables predictoras y target
X = df[['valoracion', 'atraccion', 'tiempo_de_espera','duracion']]
y = df['emocion']

In [56]:
# Si 'atraccion' es texto, la codificamos
df['atraccion'] = df['atraccion'].astype('category').cat.codes

In [57]:
# Escalar variables numéricas
from sklearn.preprocessing import StandardScaler
X = df[['valoracion', 'atraccion', 'duracion', 'tiempo_de_espera']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en train/test usando los datos escalados
X_train, X_test, y_train, y_test = train_test_split( X_scaled, y, test_size=0.20, random_state=42, stratify=y)

In [60]:
print(X_scaled)
      

[[-0.94814093  0.29516153 -0.51839234 -1.86752966]
 [ 0.32110063 -1.08961049 -2.57007786 -0.37325115]
 [-1.5827617  -0.89178591  0.11212565  0.12484168]
 ...
 [-1.5827617  -0.9906982  -2.36991342 -1.1203904 ]
 [ 0.00379024 -1.28743506 -0.1380799  -0.37325115]
 [ 0.00379024 -1.58417192 -0.23816212  0.87198093]]


KNN

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

print("🔷 KNN")
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_knn))
print("Macro F1-score:", f1_score(y_test, y_pred_knn, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn))

🔷 KNN
Accuracy: 0.2072072072072072
Balanced Accuracy: 0.14195913149461062
Macro F1-score: 0.12553678305805235

Classification Report:
               precision    recall  f1-score   support

        asco       0.00      0.00      0.00       104
     enojado       0.14      0.10      0.12       947
       feliz       0.25      0.55      0.34      1726
       miedo       0.13      0.07      0.09       981
     neutral       0.17      0.14      0.15      1189
    sorpresa       0.08      0.02      0.03       765
      triste       0.18      0.12      0.14      1170

    accuracy                           0.21      6882
   macro avg       0.14      0.14      0.13      6882
weighted avg       0.17      0.21      0.17      6882



C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [62]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=2000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print("🔷 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_lr))
print("Macro F1-score:", f1_score(y_test, y_pred_lr, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

🔷 Logistic Regression
Accuracy: 0.25079918628305725
Balanced Accuracy: 0.14285714285714285
Macro F1-score: 0.057288900690387674

Classification Report:
               precision    recall  f1-score   support

        asco       0.00      0.00      0.00       104
     enojado       0.00      0.00      0.00       947
       feliz       0.25      1.00      0.40      1726
       miedo       0.00      0.00      0.00       981
     neutral       0.00      0.00      0.00      1189
    sorpresa       0.00      0.00      0.00       765
      triste       0.00      0.00      0.00      1170

    accuracy                           0.25      6882
   macro avg       0.04      0.14      0.06      6882
weighted avg       0.06      0.25      0.10      6882



C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [63]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("🔷 Decision Tree")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_dt))
print("Macro F1-score:", f1_score(y_test, y_pred_dt, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_dt))

🔷 Decision Tree
Accuracy: 0.17189770415576866
Balanced Accuracy: 0.14131271855618582
Macro F1-score: 0.1412600124928587

Classification Report:
               precision    recall  f1-score   support

        asco       0.01      0.01      0.01       104
     enojado       0.13      0.12      0.13       947
       feliz       0.25      0.24      0.25      1726
       miedo       0.15      0.15      0.15       981
     neutral       0.17      0.18      0.17      1189
    sorpresa       0.11      0.11      0.11       765
      triste       0.17      0.18      0.17      1170

    accuracy                           0.17      6882
   macro avg       0.14      0.14      0.14      6882
weighted avg       0.17      0.17      0.17      6882



In [64]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("🔷 Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_rf))
print("Macro F1-score:", f1_score(y_test, y_pred_rf, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))

🔷 Random Forest
Accuracy: 0.1891891891891892
Balanced Accuracy: 0.1463240089459036
Macro F1-score: 0.14462653497350791

Classification Report:
               precision    recall  f1-score   support

        asco       0.03      0.01      0.02       104
     enojado       0.13      0.10      0.11       947
       feliz       0.25      0.34      0.29      1726
       miedo       0.14      0.12      0.13       981
     neutral       0.18      0.19      0.18      1189
    sorpresa       0.12      0.08      0.10       765
      triste       0.18      0.17      0.18      1170

    accuracy                           0.19      6882
   macro avg       0.15      0.15      0.14      6882
weighted avg       0.18      0.19      0.18      6882



In [28]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)

print("🔷 Gradient Boosting")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_gb))
print("Macro F1-score:", f1_score(y_test, y_pred_gb, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_gb))

🔷 Gradient Boosting
Accuracy: 0.24622180888165543
Balanced Accuracy: 0.14166350065822925
Macro F1-score: 0.06421801106157396

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       130
           1       0.12      0.00      0.00      1183
           2       0.25      0.96      0.40      2157
           3       0.12      0.00      0.01      1226
           4       0.17      0.01      0.03      1487
           5       0.05      0.00      0.00       957
           6       0.11      0.01      0.01      1462

    accuracy                           0.25      8602
   macro avg       0.12      0.14      0.06      8602
weighted avg       0.15      0.25      0.11      8602



### CONCLUSION:
## El conjunto actual de variables (valoracion, atraccion, duracion, tiempo_de_espera) no contiene suficiente señal para distinguir entre 7 emociones diferentes. Incluso modelos mas potentes como Gradient Boosting y Random Forest están solo aprendiendo a identificar la clase mayoritaria (feliz)




In [65]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

clf1 = LogisticRegression(max_iter=1000)
clf2 = RandomForestClassifier()
clf3 = KNeighborsClassifier()

voting_clf = VotingClassifier(
    estimators=[('lr', clf1), ('rf', clf2), ('knn', clf3)],
    voting='soft'  # usa promedios de probabilidad en lugar de votos duros
)

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)

In [66]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

print("🔷 Voting Ensemble")
print("Accuracy:", accuracy_score(y_test, y_pred_voting))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_voting))
print("Macro F1-score:", f1_score(y_test, y_pred_voting, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_voting))

🔷 Voting Ensemble
Accuracy: 0.1941296134844522
Balanced Accuracy: 0.14257332219283253
Macro F1-score: 0.13647589750111822

Classification Report:
               precision    recall  f1-score   support

        asco       0.00      0.00      0.00       104
     enojado       0.13      0.08      0.10       947
       feliz       0.24      0.41      0.30      1726
       miedo       0.15      0.10      0.12       981
     neutral       0.17      0.17      0.17      1189
    sorpresa       0.11      0.06      0.08       765
      triste       0.18      0.18      0.18      1170

    accuracy                           0.19      6882
   macro avg       0.14      0.14      0.14      6882
weighted avg       0.17      0.19      0.18      6882



In [67]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Modelos base
base_models = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier()),
    ('knn', KNeighborsClassifier())
]

# Meta-modelo
final_model = RandomForestClassifier()

# Stacking ensemble
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=final_model,
    cv=5,
    passthrough=False  # pon True si quieres que el meta-modelo vea también X original
)

# Entrenamiento
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(max_iter=1000)),
                               ('rf', RandomForestClassifier()),
                               ('knn', KNeighborsClassifier())],
                   final_estimator=RandomForestClassifier())

In [68]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

y_pred_stack = stacking_clf.predict(X_test)

print("🔷 Stacking Ensemble")
print("Accuracy:", accuracy_score(y_test, y_pred_stack))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_stack))
print("Macro F1-score:", f1_score(y_test, y_pred_stack, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_stack))

🔷 Stacking Ensemble
Accuracy: 0.22609706480674222
Balanced Accuracy: 0.14428855996720252
Macro F1-score: 0.11106383418854314

Classification Report:
               precision    recall  f1-score   support

        asco       0.00      0.00      0.00       104
     enojado       0.15      0.04      0.07       947
       feliz       0.25      0.71      0.37      1726
       miedo       0.13      0.03      0.05       981
     neutral       0.17      0.13      0.15      1189
    sorpresa       0.16      0.01      0.02       765
      triste       0.17      0.09      0.12      1170

    accuracy                           0.23      6882
   macro avg       0.15      0.14      0.11      6882
weighted avg       0.18      0.23      0.16      6882



C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anaco\anaconda3\envs\pontia-ml\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

### UNDERSAMPLING

In [69]:
import pandas as pd
import numpy as np

# Combinar X_train con y_train en un DataFrame
X_train_df = pd.DataFrame(X_train, columns=['valoracion', 'atraccion', 'duracion', 'tiempo_de_espera'])  # ajusta columnas
y_train_df = pd.Series(y_train, name='emocion')

df_train = pd.concat([X_train_df, y_train_df], axis=1)

# Ver la cantidad por clase
print("Distribución original:")
print(df_train['emocion'].value_counts())

# Obtener el tamaño de la clase minoritaria
min_count = df_train['emocion'].value_counts().min()

# Undersample: reducir todas las clases al mismo número de ejemplos
df_balanced = df_train.groupby('emocion', group_keys=False).apply(lambda x: x.sample(min_count, random_state=42))

# Separar nuevamente X e y
X_train_bal = df_balanced.drop('emocion', axis=1).values
y_train_bal = df_balanced['emocion'].values

print("Distribución balanceada:")
print(pd.Series(y_train_bal).value_counts())

Distribución original:
emocion
feliz       6901
neutral     4758
triste      4678
miedo       3922
enojado     3787
sorpresa    3062
asco         418
Name: count, dtype: int64
Distribución balanceada:
asco        418
enojado     418
feliz       418
miedo       418
neutral     418
sorpresa    418
triste      418
Name: count, dtype: int64


C:\Users\anaco\AppData\Local\Temp\ipykernel_34944\2850916215.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_train.groupby('emocion', group_keys=False).apply(lambda x: x.sample(min_count, random_state=42))


In [71]:
import pandas as pd

# Convertir a DataFrame para trabajar más fácilmente
df_bal = pd.DataFrame(X_train_bal, columns=['valoracion', 'atraccion', 'duracion', 'tiempo_de_espera'])
df_bal['emocion'] = y_train_bal

# Eliminar filas con NaNs
df_bal = df_bal.dropna()

# Separar de nuevo
X_train_bal = df_bal.drop('emocion', axis=1).values
y_train_bal = df_bal['emocion'].values

In [72]:
stacking_clf.fit(X_train_bal, y_train_bal)
y_pred_stack_bal = stacking_clf.predict(X_test)

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

print("🔷 Stacking Ensemble (con undersampling)")
print("Accuracy:", accuracy_score(y_test, y_pred_stack_bal))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_stack_bal))
print("Macro F1-score:", f1_score(y_test, y_pred_stack_bal, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_stack_bal))

🔷 Stacking Ensemble (con undersampling)
Accuracy: 0.14617843650101714
Balanced Accuracy: 0.13895378428487767
Macro F1-score: 0.1278656868978209

Classification Report:
               precision    recall  f1-score   support

        asco       0.01      0.13      0.02       104
     enojado       0.14      0.18      0.16       947
       feliz       0.25      0.16      0.19      1726
       miedo       0.15      0.16      0.15       981
     neutral       0.18      0.16      0.17      1189
    sorpresa       0.10      0.02      0.03       765
      triste       0.19      0.16      0.17      1170

    accuracy                           0.15      6882
   macro avg       0.14      0.14      0.13      6882
weighted avg       0.18      0.15      0.15      6882



In [73]:
df['emocion_grupo'] = df['emocion'].replace({
    'feliz': 'positiva',
    'sorpresa': 'positiva',
    'neutral': 'neutra',
    'asco': 'negativa',
    'enojado': 'negativa',
    'miedo': 'negativa',
    'triste': 'negativa'
})

In [74]:
# X: mismas variables que ya preprocesaste
# y: nueva variable objetivo
y = df['emocion_grupo']

# Codificar la variable target
from sklearn.preprocessing import LabelEncoder
le_grupo = LabelEncoder()
y_encoded = le_grupo.fit_transform(y)  # 0=negativa, 1=neutra, 2=positiva

# División train/test (usamos las mismas features y escalado si ya lo hiciste)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, stratify=y_encoded, random_state=42)

In [75]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("🔷 Random Forest con emociones agrupadas")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
print("Macro F1-score:", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le_grupo.classes_))

🔷 Random Forest con emociones agrupadas
Accuracy: 0.4075796326435713
Balanced Accuracy: 0.3333210133535041
Macro F1-score: 0.3247416000322489

Classification Report:
               precision    recall  f1-score   support

    negativa       0.46      0.58      0.51      4002
      neutra       0.18      0.08      0.11      1487
    positiva       0.37      0.34      0.36      3113

    accuracy                           0.41      8602
   macro avg       0.33      0.33      0.32      8602
weighted avg       0.38      0.41      0.39      8602



In [79]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Modelo base
rf = RandomForestClassifier(random_state=42)

# Espacio de hiperparámetros (amplio pero manejable)
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'class_weight': [None, 'balanced']
}

In [80]:
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=30,  # número de combinaciones a probar
    cv=5,
    scoring='f1_macro',  # porque buscamos equilibrio entre clases
    n_jobs=-1,
    random_state=42,
    verbose=2
)

random_search.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=30, n_jobs=-1,
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'max_depth': [None, 10, 20, 30, 50],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000024983504D30>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002498E271900>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000249DDDDFC40>},
                   random_state=42, scoring='f1_macro', verbose=2)

In [81]:
best_rf = random_search.best_estimator_
print("🔍 Mejores hiperparámetros:")
print(random_search.best_params_)

# Predicción
y_pred_best = best_rf.predict(X_test)

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report

print("\n🔷 Random Forest Optimizado (Randomized Search)")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred_best))
print("Macro F1-score:", f1_score(y_test, y_pred_best, average='macro'))
print("\nClassification Report:\n", classification_report(y_test, y_pred_best, target_names=le_grupo.classes_))

🔍 Mejores hiperparámetros:
{'class_weight': 'balanced', 'max_depth': 50, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 148}

🔷 Random Forest Optimizado (Randomized Search)
Accuracy: 0.38014415252266914
Balanced Accuracy: 0.3278741229918212
Macro F1-score: 0.32721333500282707

Classification Report:
               precision    recall  f1-score   support

    negativa       0.45      0.49      0.47      4002
      neutra       0.17      0.14      0.16      1487
    positiva       0.36      0.35      0.35      3113

    accuracy                           0.38      8602
   macro avg       0.33      0.33      0.33      8602
weighted avg       0.37      0.38      0.37      8602

